<a href="https://colab.research.google.com/github/MiHarsh/CodaLab-SharedTask/blob/main/Conventional_Approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required libraries

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-2b199d99-d0f1-fa56-c08c-dfe7c9ae4413)


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


import sys
import re
!pip install emoji --quiet
import emoji
!pip install contractions --quiet
import contractions
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import unicodedata

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 245kB 5.9MB/s 
     |████████████████████████████████| 317kB 12.4MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Dataset

In [5]:
train_datapath="https://raw.githubusercontent.com/MiHarsh/MiHarsh/master/Constraint_English_Train%20-%20Sheet1.csv"
val_datapath  ="https://raw.githubusercontent.com/MiHarsh/MiHarsh/master/Constraint_English_Val%20-%20Sheet1.csv"
train         = pd.read_csv(train_datapath)
valid         = pd.read_csv(val_datapath)
total         = pd.concat([train,valid],ignore_index=True)
mix           = total.iloc[:,1:]
mix

,tweet,label
0,The CDC currently reports 99031 deaths. In gen...,real
1,States reported 1121 deaths a small rise from ...,real
2,Politically Correct Woman (Almost) Uses Pandem...,fake
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,Populous states can generate large case counts...,real
...,...,...
8555,Donald Trump wrongly claimed that New Zealand ...,fake
8556,Current understanding is #COVID19 spreads most...,real
8557,Nothing screams “I am sat around doing fuck al...,fake
8558,Birx says COVID-19 outbreak not under control ...,fake


## Data Cleaning

In [8]:
def cleaning(text):
  text= text.lower()
  text= emoji.demojize(text)
  text=contractions.fix(text)
  text=text.strip()
  text=text.replace('[^\w\s]','')
  text=re.sub(r'http\S+', '', text)
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
  text = REPLACE_BY_SPACE_RE.sub(' ' , text)
  text = BAD_SYMBOLS_RE.sub(' ',text)
  
  return text

clean=mix['tweet'].apply(cleaning)
STOPWORDS = set(stopwords.words('english'))

ff=[]
for i in clean:
  text=unicodedata.normalize('NFKD', i).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  ff.append(text)
dd=pd.DataFrame(ff)
dataset = pd.concat([dd,mix['label']],axis=1)
dataset

,0,label
0,the cdc currently reports 99031 deaths in gen...,real
1,states reported 1121 deaths a small rise from ...,real
2,politically correct woman almost uses pandem...,fake
3,indiafightscorona we have 1524 covid testin...,real
4,populous states can generate large case counts...,real
...,...,...
8555,donald trump wrongly claimed that new zealand ...,fake
8556,current understanding is covid19 spreads most...,real
8557,nothing screams i am sat around doing fuck al...,fake
8558,birx says covid 19 outbreak not under control ...,fake


## Models for Conventional Approaches

In [9]:
regressor = LogisticRegression(solver='liblinear') 
S         = svm.SVC()
N         = BernoulliNB()
Rf        = RandomForestClassifier(max_depth=8, random_state=0)
ANN       = MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(5, 2), random_state=1)

## Count Vector

In [10]:
cv= CountVectorizer()
transform = cv.fit_transform(dataset[0].values.astype('U'))
x_train= transform[:len(train)]                                           #trainset 
x_valid= transform[len(train):]                                           #validset
print(x_train.shape)
print(x_valid.shape)

(6420, 16488)
(2140, 16488)


### Training the models

In [11]:
regressor.fit(x_train,dataset['label'][:len(train)])
S.fit(x_train,dataset['label'][:len(train)])
N.fit(x_train,dataset['label'][:len(train)])
Rf.fit(x_train, dataset['label'][:len(train)])
ANN.fit(x_train, dataset['label'][:len(train)])

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

### Prediction of labels for Valid dataset using Count Vector

In [12]:
y_pred1= regressor.predict(x_valid)
y_pred2= S.predict(x_valid)
y_pred3= N.predict(x_valid)
y_pred4= Rf.predict(x_valid)
y_pred5= ANN.predict(x_valid)

In [13]:
for i in range(5):
  ac=accuracy_score(dataset['label'][len(train):], globals()['y_pred'+str(int(i+1))])
  print(ac)
  print(classification_report(dataset['label'][len(train):], globals()['y_pred'+str(int(i+1))]))

0.9369158878504673
              precision    recall  f1-score   support

        fake       0.92      0.95      0.93      1020
        real       0.95      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

0.9210280373831776
              precision    recall  f1-score   support

        fake       0.90      0.94      0.92      1020
        real       0.95      0.90      0.92      1120

    accuracy                           0.92      2140
   macro avg       0.92      0.92      0.92      2140
weighted avg       0.92      0.92      0.92      2140

0.9112149532710281
              precision    recall  f1-score   support

        fake       0.87      0.95      0.91      1020
        real       0.95      0.87      0.91      1120

    accuracy                           0.91      2140
   macro avg       0.91      0.91      0.91      2140
weighted avg     

## Tf-IDf

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset[0])
X_train= X[:len(train)]
X_valid= X[len(train):]

In [15]:
regressor.fit(X_train,dataset['label'][:len(train)])
S.fit(X_train,dataset['label'][:len(train)])
N.fit(X_train,dataset['label'][:len(train)])
Rf.fit(X_train, dataset['label'][:len(train)])
ANN.fit(X_train, dataset['label'][:len(train)])

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [16]:
Y_pred1= regressor.predict(X_valid)
Y_pred2= S.predict(X_valid)
Y_pred3= N.predict(X_valid)
Y_pred4= Rf.predict(X_valid)
Y_pred5= ANN.predict(X_valid)

In [17]:
for i in range(5):
  ac=accuracy_score(dataset['label'][len(train):], globals()['Y_pred'+str(int(i+1))])
  print(ac)
  print(classification_report(dataset['label'][len(train):], globals()['Y_pred'+str(int(i+1))]))

0.9257009345794392
              precision    recall  f1-score   support

        fake       0.91      0.94      0.92      1020
        real       0.95      0.91      0.93      1120

    accuracy                           0.93      2140
   macro avg       0.93      0.93      0.93      2140
weighted avg       0.93      0.93      0.93      2140

0.9341121495327103
              precision    recall  f1-score   support

        fake       0.91      0.96      0.93      1020
        real       0.96      0.91      0.94      1120

    accuracy                           0.93      2140
   macro avg       0.93      0.94      0.93      2140
weighted avg       0.94      0.93      0.93      2140

0.9112149532710281
              precision    recall  f1-score   support

        fake       0.87      0.95      0.91      1020
        real       0.95      0.87      0.91      1120

    accuracy                           0.91      2140
   macro avg       0.91      0.91      0.91      2140
weighted avg     